In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../.."))

from scipy.stats import f_oneway
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import plotly.express as px
import pandas as pd
import numpy as np
from tqdm import tqdm

### Carregando os dados integralmente

In [ ]:
df_agg_cod = pd.read_excel(
    '../../datasets/base_vertical/intermediate/HS_agregado_codificado_20windows.xlsx')

### Separando em treino e validação.

In [ ]:
colunas_classes = [
    col for col in df_agg_cod.columns if col.startswith('Heart Sound Type_')]

X_total = df_agg_cod.drop(columns=colunas_classes)
y_total = df_agg_cod["Heart Sound Type_Normal"]

X_train, X_val, y_train, y_val = train_test_split(
    X_total, y_total, test_size=0.2, stratify=y_total)

X_train.assign(Heart_Sound_Type_Normal=y_train).to_excel(
    "../../datasets/base_vertical/train/train_binario.xlsx", index=False)
X_val.assign(Heart_Sound_Type_Normal=y_val).to_excel(
    "../../datasets/base_vertical/val/val_binario.xlsx", index=False)

### Normalizando os dados

In [ ]:
dummy_cols = [col for col in df_agg_cod.columns
              if df_agg_cod[col].dropna().isin([0, 1]).all() and df_agg_cod[col].nunique() <= 2]
non_dummy_cols = [col for col in df_agg_cod.columns if col not in dummy_cols]

sca = StandardScaler()

X_train_scaled = sca.fit_transform(X_train[non_dummy_cols])
X_train = pd.DataFrame(X_train_scaled, columns=non_dummy_cols)


X_val_scaled = sca.transform(X_val[non_dummy_cols])
X_val = pd.DataFrame(X_val_scaled, columns=non_dummy_cols)


X_train.to_excel(
    "../../datasets/base_vertical/train/train_binario_normalized.xlsx", index=False)
X_val.assign(Heart_Sound_Type_Normal=y_val).to_excel(
    "../../datasets/base_vertical/val/val_binario_normalized.xlsx", index=False)

In [ ]:
X_train["Heart_Sound_Type_Normal"] = y_train
X_train.corr()["Heart_Sound_Type_Normal"].sort_values(ascending=False)

Heart_Sound_Type_Normal      1.000000
variacao_curvatura_std       0.071450
comprimento_curva_std        0.059091
mudanca_media_direcao_std    0.058131
mfcc_1_mean                  0.057869
                               ...   
zero_crossing_rate_mean     -0.072901
centro_y_mean               -0.123374
rqa_fs_mean                       NaN
rqa_embedding_dim_mean            NaN
rqa_delay_mean                    NaN
Name: Heart_Sound_Type_Normal, Length: 67, dtype: float64

In [23]:
X_train[["rqa_fs_mean", "rqa_embedding_dim_mean", "rqa_delay_mean"]].describe()

,rqa_fs_mean,rqa_embedding_dim_mean,rqa_delay_mean
count,800.0,800.0,800.0
mean,0.0,0.0,0.0
std,0.0,0.0,0.0
min,0.0,0.0,0.0
25%,0.0,0.0,0.0
50%,0.0,0.0,0.0
75%,0.0,0.0,0.0
max,0.0,0.0,0.0
